In [2]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg

from ASCE import load_combos
from steel_design.AISC import aisc
from concrete_design.material_properties import elastic_modulus, rigidity_modulus
from concrete_design.steel_reinforcement import rebar_df as rebar
from masonry_design.material_properties import NW_CMU, elastic_modulus
conc_density = 150.0 #pcf
steel_density = 0.2833 #pci

display(Latex(r"\newpage")) #pagebreak

<IPython.core.display.Latex object>

# Masonry Beam Design

### Material Properties

In [48]:
fm = 1500 #psi
fr = 100 #psi

Em = elastic_modulus(fm) #ksi
Es = 29000 #ksi

In [49]:
Fs = 32000 #psi
NW_CMU

,"t=6""","t=8""","t=10""","t=12"""
Solid,63,84,104,133
"16""",52,66,86,103
"24""",48,61,78,94
"32""",47,58,74,89
"40""",46,56,72,86
"48""",45,55,70,83


### Geometry

In [50]:
t = 10 #in
b = 9.625 #in
h = 4.25*12 #in
l = 20 #ft

#### Loading

In [51]:
RDL = 204 #plf
wall_height = 4 #ft
wall_weight = NW_CMU.loc['48"', f't={t}"'] * (wall_height - h/12) \
    + NW_CMU.loc['Solid', f't={t}"'] * h/12 #plf
RLL = 240 #plf
RSL = 516 #plf

#support = "pinned-pinned"
support = "fixed-fixed"

w = load_combos.ASD(D=(RDL+wall_weight), Lr=RLL, S=RSL)
f"w = {round(w, 2)} plf"

'w = 1144.5 plf'

__Moment:__

In [52]:
if support == "pinned-pinned":
    M = w * l**2 / 8 / 1000 #kip-ft
elif support == "fixed-fixed":
    M = w * l**2 / 12 / 1000 #kip-ft
    
f"M = {round(M, 2)} kip-ft"

'M = 38.15 kip-ft'

__Shear:__

In [53]:
V = w * l / 2 / 1000 #kips
f"V = {round(V, 2)} kips"

'V = 11.44 kips'

__Axial Compression:__

In [54]:
P = 0

### Reinforcement
#### Flexural Reinforcement

In [55]:
bar = "#6"
no_bars = 4
As = no_bars * rebar.loc[bar, "Area"]
d = h - 3 #in
rho = As / (b*d)
n = Es / Em
k = sqrt(2*rho*n + (rho*n)**2) - rho*n
j = 1 - k/3
f"As={As}, d={d}, rho={round(rho, 5)}, n={round(n, 3)}, j={round(j, 4)}, k={round(k, 4)}"

'As=1.76, d=48.0, rho=0.00381, n=21.481, j=0.8897, k=0.3309'

#### Shear Reinforcement

In [56]:
bar = "#3"
no_legs = 1
Av = no_legs * rebar.loc[bar, "Area"]
f"Av = {Av} in^2"

'Av = 0.11 in^2'

## ASD Moment Design

In [57]:
Fb = 0.45 * fm #psi

Mm = Fb * b * d**2 * (j*k/2) / 1000/12 #kip-ft
print(f"Mm = {round(Mm, 2)} kip-ft")

Ms = As * Fs * j * d / 1000/12 #kip-ft
print(f"Ms = {round(Ms, 2)} kip-ft")

Mm = 183.63 kip-ft
Ms = 200.43 kip-ft


In [58]:
Ma = min(Ms, Mm)
f"Ma = {round(Ma, 2)} kip-ft"

'Ma = 183.63 kip-ft'

In [59]:
f"Flexure DCR = {round(M / Ma, 3)}"

'Flexure DCR = 0.208'

## ASD Shear Design

In [104]:
fv = V*1000 / Anv
print(f"shear stress, fv = {round(fv, 2)} psi")

shear stress, fv = 24.77 psi


In [105]:
dv = d #in
Anv = b * dv #in^2
gamma_g = 1.0

MV_ratio = M*12/(V*dv)
if MV_ratio <= 0.25:
    Fv_base = 3 * sqrt(fm) * gamma_g
elif MV_ratio >= 1.0:
    Fv_base = 2 * sqrt(fm) * gamma_g
else:
    Fv_base = np.interp([MV_ratio], [0.25, 1.0], [3*sqrt(fm)*gamma_g, 2*sqrt(fm)*gamma_g])[0]
f"Fv,base = {round(Fv_base, 2)} psi"

'Fv,base = 86.07 psi'

In [106]:
Fvm = max(0.5 * ((4.0 - 1.75*MV_ratio) * sqrt(fm)) + 0.25*P/(b*h), 0) #psi
f"Fvm = {round(Fvm, 2)} psi"

'Fvm = 49.22 psi'

In [107]:
s_v = 24 #in
Fvs = Av * Fs * dv / (2 * Anv * s_v) #psi
f"Fvs = {round(Fvs, 2)} psi"

'Fvs = 7.62 psi'

In [108]:
Fv = min((Fvm + Fvs) * gamma_g, Fv_base)
f"Fv = {round(Fv, 2)} psi"

'Fv = 56.84 psi'

In [109]:
DCR_v = fv/Fv
print(f"DCR_v = {round(DCR_v, 3)}")

DCR_v = 0.436


## Deflection

#### Deflections

In [ ]:
Ig = b * h**3 / 12 #in^4
print(f"Ig = {round(Ig, 2)} in^4")

Sg = b * h**2 / 6 #in^3
Mcr = fr * Sg #in-lbs

Icr = (b*(k*d)**3)/3 + (n*rho*b*d)*(d - k*d)**2
print(f"Icr = {round(Icr, 2)} in^4")

Ma = Ma * 1000 * 12 #in-lbs
Ieff = min(Ig * (Mcr / Ma)**3 + Icr * (1 - (Mcr / Ma)**3), Ig)
print(f"Ieff = {round(Ieff, 2)} in^4")

In [ ]:
if support == "pinned-pinned":
    defl = 5 * (w/12) * (l*12)**4 / (384 * (Em*1000) * Ieff)
elif support == "fixed-fixed":
    defl = (w/12) * (l*12)**4 / (384 * (Em*1000) * Ieff)
    
f"deflection = {round(defl, 4)} in"

In [ ]:
defl_limit = l*12 / 480
f"deflection limit = {round(defl_limit, 4)} in"

In [ ]:
defl < defl_limit